In [2]:
import math
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Dropout, Activation
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import seaborn as sns

2022-10-15 23:34:46.457638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 23:34:46.756965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 23:34:46.756979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-15 23:34:46.793930: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 23:34:47.657142: W tensorflow/stream_executor/platform/de

In [4]:
initial_df = pd.read_csv('ETHBTC-1m.csv')

In [5]:
initial_df.head(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,1609459200000,0.025464,0.025464,0.025447,0.025449,625.182,1609459259999,15.915061,188,225.278,5.734506,0
1,1609459260000,0.025449,0.025479,0.025435,0.025465,243.229,1609459319999,6.193560,171,90.259,2.298146,0
2,1609459320000,0.025465,0.025475,0.025447,0.025454,195.633,1609459379999,4.980014,107,66.517,1.693127,0
3,1609459380000,0.025453,0.025463,0.025442,0.025443,135.177,1609459439999,3.440634,87,59.216,1.507130,0
4,1609459440000,0.025443,0.025450,0.025427,0.025436,308.023,1609459499999,7.834913,115,187.253,4.762954,0


In [6]:
initial_df.describe()

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
count,9.389100e+05,938910.000000,938910.000000,938910.000000,938910.000000,938910.000000,9.389100e+05,938910.000000,938910.000000,938910.000000,938910.000000,938910.0
mean,1.637672e+12,0.063267,0.063292,0.063241,0.063267,119.395209,1.637672e+12,6.572083,183.570057,58.873810,3.246211,0.0
std,1.627950e+10,0.014759,0.014760,0.014758,0.014759,185.799018,1.627950e+10,9.480170,224.754577,99.023891,5.115744,0.0
min,1.609459e+12,0.023090,0.023132,0.023083,0.023090,0.000000,1.609459e+12,0.000000,0.000000,0.000000,0.000000,0.0
25%,1.623579e+12,0.058257,0.058285,0.058230,0.058259,30.567500,1.623579e+12,2.089847,75.000000,13.409000,0.908464,0.0
50%,1.637686e+12,0.067978,0.068002,0.067953,0.067978,63.678800,1.637686e+12,3.947388,125.000000,30.172700,1.863984,0.0
75%,1.651770e+12,0.073310,0.073333,0.073287,0.073309,138.833000,1.651770e+12,7.514529,213.000000,67.370000,3.697792,0.0
max,1.665853e+12,0.088330,0.088366,0.088306,0.088332,21933.475900,1.665853e+12,1308.407466,16517.000000,10053.086000,554.255723,0.0


In [7]:
def time_features(df):
    df["utc"] = df.open_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df['minute'] = df.utc.apply(lambda time: time.minute)
    df['hour'] = df.utc.apply(lambda time: time.hour)
    df['day'] = df.utc.apply(lambda time: time.day)
    df["weekday"] = df.utc.apply(lambda time: time.dayofweek)
    df['month'] = df.utc.apply(lambda time: time.month)
    df['year'] = df.utc.apply(lambda time: time.year).astype("category")
    return df

In [8]:
initial_df = time_features(initial_df)

In [9]:
create_feature_dict = {
    'open': [np.sum, np.std, np.max],
}

df = initial_df.groupby([initial_df.year, initial_df.month, initial_df.day, initial_df.hour]).agg(create_feature_dict).dropna()

In [12]:
df[[('open',  'sum'), ('open',  'std'), ('open', 'amax')]]

open                    
                          sum       std      amax
year month day hour                              
2021 1     1   0     1.522885  0.000035  0.025465
               1     1.521054  0.000076  0.025503
               2     1.526385  0.000029  0.025510
               3     1.528150  0.000024  0.025517
               4     1.524831  0.000017  0.025444
...                       ...       ...       ...
2022 10    15  13    4.024882  0.000057  0.067266
               14    4.035776  0.000032  0.067331
               15    4.035887  0.000037  0.067337
               16    4.033420  0.000049  0.067319
               17    0.201590  0.000002  0.067198

[15652 rows x 3 columns]

In [13]:
data = df.copy()

# Get the number of rows in the data
nrows = df.shape[0]

# Convert the data to numpy values
np_data_unscaled = np.array(df)
np_data_unscaled = np.reshape(np_data_unscaled, (nrows, -1))
print(np_data_unscaled.shape)

# Transform the data by scaling each feature to a range between 0 and 1
scaler = RobustScaler()
np_data_scaled = scaler.fit_transform(np_data_unscaled)

# Set the sequence length - this is the timeframe used to make a single prediction
sequence_length = 110

# Prediction Index
index_Close = 0

# Split the training data into train and train data sets
# As a first step, we get the number of rows to train the model on 80% of the data 
train_data_len = math.ceil(np_data_scaled.shape[0] * 0.8)

# Create the training and test data
train_data = np_data_scaled[0:train_data_len, :]
test_data = np_data_scaled[train_data_len - sequence_length:, :]

# The RNN needs data with the format of [samples, time steps, features]
# Here, we create N samples, sequence_length time steps per sample, and 6 features
def partition_dataset(sequence_length, data):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        x.append(data[i-sequence_length:i,:]) #contains sequence_length values 0-sequence_length * columsn
        y.append(data[i, index_Close]) #contains the prediction values for validation (3rd column = Close),  for single-step prediction
    
    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y

# Generate training data and test data
x_train, y_train = partition_dataset(sequence_length, train_data)
x_test, y_test = partition_dataset(sequence_length, test_data)

# Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Validate that the prediction value and the input match up
# The last close price of the second input sample should equal the first prediction value
print(x_test[1][sequence_length-1][index_Close])
print(y_test[0])

(15652, 3)
(12412, 110, 3) (12412,)
(3130, 110, 3) (3130,)
-0.5507786497504518
-0.5507786497504518


In [14]:
# Configure the neural network model
epochs = 12; batch_size = 1;

# Model with n_neurons = inputshape Timestamps, each with x_train.shape[2] variables
n_neurons = x_train.shape[1] * x_train.shape[2]
model = Sequential()
model.add(LSTM(n_neurons, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(n_neurons, return_sequences=False))
model.add(Dense(5))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mean_squared_error")

2022-10-15 23:36:42.856193: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-15 23:36:42.856222: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedorovn19-Nitro-AN515-45): /proc/driver/nvidia/version does not exist
2022-10-15 23:36:42.857247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
